In [57]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, auc, roc_curve

df = pd.read_csv('kaggle_data/train.csv')
df.drop(columns=['Id'], inplace=True)
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
1,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
2,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
3,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
4,0,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0


In [58]:
# DIVIDIMOS TRAIN TEST
from sklearn.model_selection import train_test_split 

X = df.drop(['SeriousDlqin2yrs'], axis=1)
y = df['SeriousDlqin2yrs']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.20,
                                                    random_state=42)




In [59]:
imputer = SimpleImputer()
imputer.fit(X_train)

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [60]:
# MODELO
log_reg = LogisticRegression(max_iter = 5000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [61]:
# PREDICCION
predictions = log_reg.predict(X_test)
predictions_proba = log_reg.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, predictions_proba)

print("Accuracy:", accuracy_score(y_test, predictions))
print("AUC:", auc(fpr, tpr))

Accuracy: 0.9369304899575402
AUC: 0.7055307417236129


### Submission file

In [62]:
# DATOS DE TEST
df_submission = pd.read_csv('kaggle_data/test.csv')
df_submission.head()

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,9,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0
2,17,0.166284,53,0,0.188274,8800.0,7,0,0,0,0.0
3,18,0.221813,43,0,0.527888,3280.0,7,0,1,0,2.0
4,24,0.046560,58,0,0.241622,2416.0,9,0,1,0,0.0


In [63]:
# LIMPIEZA
test_data = imputer.transform(df_submission.drop(columns = ['Id']))

In [64]:
# Predictions
preds_submission = log_reg.predict_proba(test_data)[:, 1]

submission = df_submission.copy()[['Id']]
submission['SeriousDlqin2yrs'] = preds_submission
submission.head()

,Id,SeriousDlqin2yrs
0,0,0.171891
1,9,0.023479
2,17,0.039885
3,18,0.084490
4,24,0.042520


In [65]:
# Exportamos el submission file
submission.to_csv('submission_pruebas.csv', index=False)